In [11]:
import numpy as np 
import os 
import h5py
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from csv import reader
from sklearn.preprocessing import scale
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout 
from tensorflow.keras import models
from sklearn.metrics import confusion_matrix

In [12]:
def extract_data(source, kind, reshape, standardize):
    train_imgs, train_vals = extract_helper(source, 'train', kind, reshape)
    test_imgs, test_vals = extract_helper(source, 'valid', kind, reshape)
    if standardize == True:
        mean = np.mean(train_imgs, axis = (1,2), keepdims = True)
        std = np.std(train_imgs, axis = (1,2), keepdims = True)
        train_imgs = (train_imgs - mean)/std
        test_imgs = (test_imgs - mean)/std
    return train_imgs, train_vals, test_imgs, test_vals

def extract_helper(source, torv, kind, reshape):
    os.chdir(source+'\\MURA-v1.1')
    os.chdir(torv+'_specific_paths')
    file = open(torv+'_image_paths_'+kind+'.csv')
    return extract(source, file, reshape)
    
def extract(source, file, reshape):
    readCSV = reader(file)
    imgs = []
    vals = []
    for row in readCSV:
        im = cv2.imread(source+'\\'+row[0], cv2.IMREAD_GRAYSCALE)
        imgs.append(np.array(cv2.resize(im,reshape)))
        if 'positive' in row[0]:
            vals.append(1)
        else:
            vals.append(0)
    file.close()
    imgs = np.array(imgs)
    vals = np.array(vals)
    imgs = np.expand_dims(imgs, axis=3)
    return imgs,vals

In [13]:
def patient_code(path):
    pos = path.find('patient')+7
    return path[pos:pos+5]

def patient_value(path):
    if 'positive' in path:
        return 1
    return 0

class patient:
    def __init__(self, imgs, vals, value):
        self.imgs = imgs
        self.vals = vals
        self.value = value
        
def extract_data_patients(source, kind, reshape, standardize):
    train_patients = extract_helper_patients(source, 'train', kind, reshape)
    test_patients  = extract_helper_patients(source, 'valid',  kind, reshape)
    if standardize == True:
        train_imgs = np.array([])
        for p in train_patients:
            train_imgs = np.concatenate(train_imgs, p.imgs)
        mean = np.mean(train_imgs, axis = (1,2), keepdims = True)
        std = np.std(train_imgs, axis = (1,2), keepdims = True)
        for p in train_patients:
            p.imgs = (p.imgs - mean)/std
        for p in test_patients:
            p.imgs = (p.imgs - mean)/std
    return train_patients, test_patients
    
def extract_helper_patients(source, torv, kind, reshape):
    os.chdir(source+'\\MURA-v1.1')
    os.chdir(torv+'_specific_paths')
    file = open(torv+'_image_paths_'+kind+'.csv')
    return extract_patients(source, file, reshape)

def extract_patients(source, file, reshape):
    patients = []
    readCSV = reader(file)
    imgs = []
    vals = []
    
    row = next(readCSV)
    prev_patient = patient_code(row[0])
    vals.append(patient_value(row[0]))
    im = cv2.imread(source+'\\'+row[0], cv2.IMREAD_GRAYSCALE)
    imgs.append(np.array(cv2.resize(im,reshape)))
    
    for row in readCSV:
        curr_patient = patient_code(row[0])
        if curr_patient == prev_patient:
            vals.append(patient_value(row[0]))
            im = cv2.imread(source+'\\'+row[0], cv2.IMREAD_GRAYSCALE)
            imgs.append(np.array(cv2.resize(im,reshape)))
                
        else:
            imgs = np.array(imgs)
            imgs = np.expand_dims(imgs, axis=3)
            vals = np.array(vals)
            patients.append(patient(imgs, vals, vals[0]))
            imgs = []
            vals = []
            prev_patient = curr_patient
            vals.append(patient_value(row[0]))
            im = cv2.imread(source+'\\'+row[0], cv2.IMREAD_GRAYSCALE)
            imgs.append(np.array(cv2.resize(im,reshape)))
                
    file.close()
    imgs = np.array(imgs)
    imgs = np.expand_dims(imgs, axis=3)
    vals = np.array(vals)
    patients.append(patient(imgs, vals, vals[0])) 

    return patients  

In [14]:
def classic_validation(model, data_x, data_y, batch_size, number_of_epochs, rate):
    rate = int(len(data_y)*rate)
    train_x, train_y = shuffler(data_x[:rate], data_y[:rate])
    valid_x, valid_y = shuffler(data_x[rate:], data_y[rate:])
    model.fit(train_x, train_y, batch_size = batch_size, epochs = number_of_epochs)
    score = model.evaluate(valid_x,valid_y)[1]
    model.fit(valid_x, valid_y, batch_size = batch_size, epochs = number_of_epochs)
    return score, model

def k_fold_cross_validation(k, model, data_x, data_y, batch_size, number_of_epochs):
    data_x, data_y = shuffler(data_x, data_y)
    folds_x = []
    folds_y = []
    l = len(data_y)
    for i in range(k):
        folds_x[i] = data_x[(l//k)*i: (l//k)*(i+1)]
        folds_y[i] = data_y[(l//k)*i: (l//k)*(i+1)]
    score = 0
    for i in range(k):
        model_copy = models.clone_model(model)
        for j in range(k):
            if j!=i:
                model_copy.fit(folds_x[j],folds_y[j], batch_size = batch_size, epochs = number_of_epochs)
        score += model_copy.evaluate(folds_x[i],folds_y[i])[1]
    model = mode_copy
    model.fit(folds_x[k-1],folds_y[k-1], batch_size = batch_size, epochs = number_of_epochs)
    return score/k, model

def shuffler(data_x, data_y):
    p = np.random.permutation(len(data_y))
    return (data_x[p], data_y[p])

def conf_matrix(model, data_x, data_y): # TN, FP, FN, TP
    y_pred = model.predict(data_x).flatten().tolist()
    y_true = data_y.tolist()
    for i in range(len(y_pred)):
        y_pred[i] = round(y_pred[i])
    return confusion_matrix(y_true, y_pred) 

def patients_conf_matrix(model, test_patients): # TN, FP, FN, TP
    y_true = []
    y_pred = []
    for p in test_patients:
        y_true.append(p.value)
        p_predict = round(np.mean(model.predict(p.imgs)))
        y_pred.append(p_predict)
    return confusion_matrix(y_true, y_pred)      

def save_model(model, source, name):
    os.chdir(source+'\\MURA-v1.1\\models')
    model.save(name+'.h5')
    
def load_model(source, name):
    os.chdir(source+'\\MURA-v1.1\\models')
    return models.load_model(name+'.h5')    

def class_weight(data_y):
    positive = np.sum(data_y)
    negative = np.size(data_y) - positive
    return {0 : 1 + positive/negative, 1: 1 + negative/positive}    

In [7]:
# 'conv3' model 

model = models.Sequential()

model.add(Conv2D(2, (3, 3), activation='relu', input_shape=(512,512,1), padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(2, (3, 3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(4, (5, 5), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(4, (5, 5), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(8, (7, 7), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
source = 'D:\\python'

print('FOREARM:')
train_x, train_y, test_x, test_y = extract_data(source ,'forearm', (512,512), False)
train_x, train_y = shuffler(train_x, train_y)
test_x, test_y = shuffler(test_x, test_y)
train_patients, test_patients = extract_data_patients(source ,'forearm', (512,512), False)

model = load_model(source, 'conv3')
model.fit(train_x, train_y, batch_size = 8, epochs = 4, class_weight = class_weight(train_y))
save_model(model, source, 'conv3')  

print(conf_matrix(model, test_x, test_y)) 
print(patients_conf_matrix(model, test_patients))

print('HUMERUS:')
train_x, train_y, test_x, test_y = extract_data(source ,'humerus', (512,512), False)
train_x, train_y = shuffler(train_x, train_y)
test_x, test_y = shuffler(test_x, test_y)
train_patients, test_patients = extract_data_patients(source ,'humerus', (512,512), False)

model = load_model(source, 'conv3')
model.fit(train_x, train_y, batch_size = 8, epochs = 4, class_weight = class_weight(train_y))
save_model(model, source, 'conv3')  

print(conf_matrix(model, test_x, test_y)) 
print(patients_conf_matrix(model, test_patients))

print('ELBOW:')
train_x, train_y, test_x, test_y = extract_data(source ,'elbow', (512,512), False)
train_x, train_y = shuffler(train_x, train_y)
test_x, test_y = shuffler(test_x, test_y)
train_patients, test_patients = extract_data_patients(source ,'elbow', (512,512), False)

model = load_model(source, 'conv3')
model.fit(train_x, train_y, batch_size = 8, epochs = 4, class_weight = class_weight(train_y))
save_model(model, source, 'conv3')  

print(conf_matrix(model, test_x, test_y)) 
print(patients_conf_matrix(model, test_patients))

print('FINGER:')
train_x, train_y, test_x, test_y = extract_data(source ,'finger', (512,512), False)
train_x, train_y = shuffler(train_x, train_y)
test_x, test_y = shuffler(test_x, test_y)
train_patients, test_patients = extract_data_patients(source ,'finger', (512,512), False)

model = load_model(source, 'conv3')
model.fit(train_x, train_y, batch_size = 8, epochs = 4, class_weight = class_weight(train_y))
save_model(model, source, 'conv3')  

print(conf_matrix(model, test_x, test_y)) 
print(patients_conf_matrix(model, test_patients))

Epoch 1/4
  96/1825 [>.............................] - ETA: 1:28 - loss: 0.6837 - acc: 0.5729

KeyboardInterrupt: 